In [1]:
from pyspark.sql import SparkSession
import pyspark

In [2]:
spark = SparkSession.builder.appName('App').getOrCreate()

In [16]:
spark = SparkSession.builder.appName("example").config("spark.driver.memory", "4g").getOrCreate()

In [17]:
spark

In [18]:
df = spark.read.csv(r'C:\Users\Usuario\Desktop\Universidad\Master\Big_Data\Spark_Work\1987.csv', header=True, inferSchema=True)

In [19]:
airports_df = spark.read.csv(r'C:\Users\Usuario\Desktop\Universidad\Master\Big_Data\Spark_Work\airports.csv', header=True, inferSchema=True)

In [20]:
carrier_df = spark.read.csv(r'C:\Users\Usuario\Desktop\Universidad\Master\Big_Data\Spark_Work\carriers.csv', header=True, inferSchema=True)

In [21]:
# Remove Forbidden variables
forbidden_variables = ['ArrTime', 'ActualElapsedTime', 'AirTime', 'TaxiIn', 'Diverted', 'CarrierDelay', 'WeatherDelay',
                   'NASDelay', 'SecurityDelay', 'LateAircraftDelay']

df = df.drop(*forbidden_variables)

In [22]:
from pyspark.sql.functions import col
# Convert columns from string to integer
df = df.withColumn("DepTime", col("DepTime").cast("int"))
df = df.withColumn("TailNum", col("TailNum").cast("int"))
df = df.withColumn("Distance", col("Distance").cast("int"))
df = df.withColumn("TaxiOut", col("TaxiOut").cast("int"))
df = df.withColumn("CancellationCode", col("CancellationCode").cast("int"))
df = df.withColumn("ArrDelay", col("ArrDelay").cast("int"))
df = df.withColumn("DepDelay", col("DepDelay").cast("int"))

In [ ]:
# Count null values
null_counts = df.selectExpr(*(f'sum(int(`{colname}` is null)) as `{colname}`' for colname in df.columns))

In [ ]:
null_counts.show()

In [23]:
# Remove columns with too many nulls
columns_to_drop = ['TailNum','TaxiOut','CancellationCode']

df = df.drop(*columns_to_drop)

In [ ]:
from pyspark.ml.feature import StringIndexer

In [ ]:
## Airport transformation to numerical

In [ ]:
indexer_origin = StringIndexer(inputCol="iata", outputCol="iataIndex")

In [ ]:
airports_df = indexer_origin.fit(airports_df).transform(airports_df)

In [ ]:
airports_df = airports_df.withColumn("iataIndex", col("iataIndex").cast("int"))

In [ ]:
# Eliminar columnas con muchos valores nulos
columns_to_drop = ['airport','city','state','country','lat','long']
airports_df = airports_df.drop(*columns_to_drop)

In [ ]:
# Perform left join on the origin
df_joined = df.join(airports_df, col("origin") == col("iata"), "left_outer")
df_joined = df_joined.withColumnRenamed("iataIndex", "iata_origin")
df_joined = df_joined.drop('iata')
# Perform left join on the destination
df_joined = df_joined.join(airports_df, col("dest") == col("iata"), "left_outer")
df_joined = df_joined.withColumnRenamed("iataIndex", "iata_dest")
df_joined = df_joined.drop(*['iata','Origin','Dest'])

In [ ]:
## Carrier transformation to numerical

In [ ]:
carrier_df = carrier_df.drop('Description')
indexer_Code = StringIndexer(inputCol="Code", outputCol="CodeIndex")
carrier_df = indexer_Code.fit(carrier_df).transform(carrier_df)
carrier_df = carrier_df.withColumn("CodeIndex", col("CodeIndex").cast("int"))
df_joined = df_joined.join(carrier_df, col("UniqueCarrier") == col("Code"), "left_outer")
df_joined = df_joined.drop(*['Code','UniqueCarrier'])

In [ ]:
df_joined.dtypes

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Compute the correlation matrix
correlation_matrix = df_joined.toPandas().corr()

# Plot the correlation matrix using seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", square=True)
plt.title("Correlation Matrix")
plt.show()

In [ ]:
# Visualize data distribution
df_joined.toPandas().hist(figsize=(12, 10), bins=20)
plt.show()

Year isn't an important variable. Month?

In [ ]:
### Train model

In [24]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator

In [25]:
df_joined = df.alias("df_copy")

In [26]:
# Drop any NaN values
df_joined = df_joined.dropna()

In [13]:
# Prepare the features using VectorAssembler
feature_columns = ['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime', 'CRSArrTime',
                    'FlightNum', 'CRSElapsedTime', 'DepDelay', 'Distance', 'Cancelled', 'iata_origin', 'iata_dest', 'CodeIndex']

vector_assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df_train = vector_assembler.transform(df_joined)

# Split the dataset into training and testing sets
(training_data, testing_data) = df_train.randomSplit([0.8, 0.2], seed=123)

# Create a Linear Regression model
lr = LinearRegression(featuresCol="features", labelCol="ArrDelay")

# Train the model
lr_model = lr.fit(training_data)

# Make predictions on the testing data
predictions = lr_model.transform(testing_data)

IllegalArgumentException: iata_origin does not exist. Available: Year, Month, DayofMonth, DayOfWeek, DepTime, CRSDepTime, CRSArrTime, UniqueCarrier, FlightNum, CRSElapsedTime, ArrDelay, DepDelay, Origin, Dest, Distance, Cancelled

In [ ]:
# Evaluate the model
evaluator = RegressionEvaluator(labelCol="ArrDelay", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)

# Print the Root Mean Squared Error (RMSE)
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Print the coefficients and intercept
print(f"Coefficients: {lr_model.coefficients}")
print(f"Intercept: {lr_model.intercept}")

Pruebas con el linear regression

In [ ]:
df_joined.show()

In [27]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

# Assuming 'spark' is your SparkSession and 'df' is your PySpark DataFrame
df_joined = df_joined.withColumn('DepTime_numeric', udf(lambda x: int(x[:2]) * 60 + int(x[2:]), IntegerType())('DepTime'))

In [28]:
# Prepare the features using VectorAssembler
feature_columns = ['DepTime','DepDelay', 'Distance']

vector_assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df_train = vector_assembler.transform(df_joined)

# Split the dataset into training and testing sets
(training_data, testing_data) = df_train.randomSplit([0.8, 0.2], seed=123)

# Create a Linear Regression model
lr = LinearRegression(featuresCol="features", labelCol="ArrDelay")

# Train the model
lr_model = lr.fit(training_data)

# Make predictions on the testing data
predictions = lr_model.transform(testing_data)

Py4JJavaError: An error occurred while calling o268.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 14 in stage 14.0 failed 1 times, most recent failure: Lost task 14.0 in stage 14.0 (TID 72) (DESKTOP-13GUISK.mshome.net executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:164)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:81)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:131)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:855)
	at org.apache.spark.rdd.RDD.$anonfu

In [ ]:
# Evaluate the model
evaluator = RegressionEvaluator(labelCol="ArrDelay", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)

# Print the Root Mean Squared Error (RMSE)
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Print the coefficients and intercept
print(f"Coefficients: {lr_model.coefficients}")
print(f"Intercept: {lr_model.intercept}")